In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/fine_tune_myself"

/content/drive/My Drive/fine_tune_myself


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 23 21:38:45 2019

@author: ghrit
"""
from keras.utils import plot_model
from keras.optimizers import SGD
from os import listdir
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense,Flatten,Dropout
from keras.models import Sequential, Model
from pickle import load
from keras.layers import Input



def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
def tr_genr(directory, targets,dset):
    while 1:
        for name in dset:
            filename = directory + '/' + name + ".jpg"
            image = load_img(filename, target_size=(224, 224))
            image = img_to_array(image)
            image= image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image= preprocess_input(image)
            y = np.array(targets[name]).reshape(1,300)
            x = np.array(image)
            yield [x,y]

# load a pre-defined list of photo identifiers
            
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return list(set(dataset))

def load_targets(fn,dset):
    targets = load(open(fn, 'rb'))
    # filter features
    targets = {k: targets[k] for k in dset}
    return targets
  
  
def get_initialized_vgg(gen, old_model):
    old_model.layers.pop()
    x= old_model.layers[-1].output

    final = Dense(300, activation='sigmoid')(x)
    model = Model(inputs= old_model.input, outputs= final)
    plot_model(model, to_file='vgg2model.png',show_shapes=True)
    # freeze pre-trained model area's layer
    for layer in old_model.layers:
        layer.trainable = False

    # update the weight that are added
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model.fit_generator(gen,steps_per_epoch=6000)
    return model
  
  
def train_vgg(gen,te_gen,model):
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers[:21]:
        layer.trainable = False
    for layer in model.layers[21:]:
        layer.trainable = True
        
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9)
    model.compile(optimizer=sgd, loss='binary_crossentropy',  
                  metrics=['binary_accuracy','categorical_accuracy'])    
    history = model.fit_generator(gen,steps_per_epoch=6000, epochs=5,
                                  validation_data=te_gen,validation_steps=1000)
    return model  ,history  



Using TensorFlow backend.


In [ ]:
directory='/content/drive/My Drive/Multi-label-Inception-net/image/images'
input_tensor = Input(shape=(224,224,3))
vggmodel = VGG16(weights='imagenet', include_top=True,input_tensor=input_tensor)
plot_model(vggmodel,to_file='vgg_orig.png',show_shapes=True)
'''
train = load_set('Flickr_8k.trainImages.txt')
tr_targets=load_targets("targets300.pkl",train[:])
tr_gen=tr_genr(directory,tr_targets,train[:])

test =load_set('Flickr_8k.testImages.txt')
te_targets=load_targets("targets300.pkl",test)
te_gen=tr_genr('/content/drive/My Drive/Multi-label-Inception-net/testim',te_targets,test)'''

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 6s 0us/step


'\ntrain = load_set(\'Flickr_8k.trainImages.txt\')\ntr_targets=load_targets("targets300.pkl",train[:])\ntr_gen=tr_genr(directory,tr_targets,train[:])\n\ntest =load_set(\'Flickr_8k.testImages.txt\')\nte_targets=load_targets("targets300.pkl",test)\nte_gen=tr_genr(\'/content/drive/My Drive/Multi-label-Inception-net/testim\',te_targets,test)'

In [ ]:
model = get_initialized_vgg(tr_gen, vggmodel)


Epoch 1/1
6000/6000 [==============================] - 189s 31ms/step - loss: 0.3159


In [ ]:
model,hist= train_vgg(tr_gen,te_gen,model)

Epoch 1/5
6000/6000 [==============================] - 256s 43ms/step - loss: 0.1266 - binary_accuracy: 0.9682 - categorical_accuracy: 0.3500 - val_loss: 0.1094 - val_binary_accuracy: 0.9676 - val_categorical_accuracy: 0.3600
Epoch 2/5
6000/6000 [==============================] - 253s 42ms/step - loss: 0.0899 - binary_accuracy: 0.9729 - categorical_accuracy: 0.4207 - val_loss: 0.1072 - val_binary_accuracy: 0.9681 - val_categorical_accuracy: 0.3710
Epoch 3/5
6000/6000 [==============================] - 253s 42ms/step - loss: 0.0779 - binary_accuracy: 0.9760 - categorical_accuracy: 0.4345 - val_loss: 0.1068 - val_binary_accuracy: 0.9683 - val_categorical_accuracy: 0.3720
Epoch 4/5
6000/6000 [==============================] - 251s 42ms/step - loss: 0.0692 - binary_accuracy: 0.9785 - categorical_accuracy: 0.4472 - val_loss: 0.1069 - val_binary_accuracy: 0.9683 - val_categorical_accuracy: 0.3800
Epoch 5/5
6000/6000 [==============================] - 251s 42ms/step - loss: 0.0623 - binary_ac

In [ ]:
model.save("ftv3004096_1.h5")

In [ ]:
import keras
print(keras.__version__)

2.2.4


In [ ]:
model,hist= train_vgg(tr_gen,te_gen,model)

Epoch 1/5
6000/6000 [==============================] - 251s 42ms/step - loss: 0.0564 - binary_accuracy: 0.9826 - categorical_accuracy: 0.4533 - val_loss: 0.1078 - val_binary_accuracy: 0.9678 - val_categorical_accuracy: 0.3710
Epoch 2/5
6000/6000 [==============================] - 250s 42ms/step - loss: 0.0512 - binary_accuracy: 0.9843 - categorical_accuracy: 0.4533 - val_loss: 0.1085 - val_binary_accuracy: 0.9678 - val_categorical_accuracy: 0.3650
Epoch 3/5
6000/6000 [==============================] - 250s 42ms/step - loss: 0.0466 - binary_accuracy: 0.9859 - categorical_accuracy: 0.4508 - val_loss: 0.1093 - val_binary_accuracy: 0.9678 - val_categorical_accuracy: 0.3660
Epoch 4/5
6000/6000 [==============================] - 250s 42ms/step - loss: 0.0425 - binary_accuracy: 0.9873 - categorical_accuracy: 0.4480 - val_loss: 0.1103 - val_binary_accuracy: 0.9678 - val_categorical_accuracy: 0.3680
Epoch 5/5
6000/6000 [==============================] - 250s 42ms/step - loss: 0.0388 - binary_ac

In [ ]:
model.save("ftv3004096_2.h5")

In [ ]:
model,hist= train_vgg(tr_gen,te_gen,model)

Epoch 1/5
6000/6000 [==============================] - 251s 42ms/step - loss: 0.0355 - binary_accuracy: 0.9897 - categorical_accuracy: 0.4493 - val_loss: 0.1123 - val_binary_accuracy: 0.9676 - val_categorical_accuracy: 0.3660
Epoch 2/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0325 - binary_accuracy: 0.9909 - categorical_accuracy: 0.4500 - val_loss: 0.1137 - val_binary_accuracy: 0.9676 - val_categorical_accuracy: 0.3750
Epoch 3/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0297 - binary_accuracy: 0.9919 - categorical_accuracy: 0.4485 - val_loss: 0.1151 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3800
Epoch 4/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0272 - binary_accuracy: 0.9928 - categorical_accuracy: 0.4493 - val_loss: 0.1165 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3810
Epoch 5/5
6000/6000 [==============================] - 249s 41ms/step - loss: 0.0250 - binary_ac

In [ ]:
model.save("ftv3004096_3.h5")

In [ ]:
model,hist= train_vgg(tr_gen,te_gen,model)

Epoch 1/5
6000/6000 [==============================] - 252s 42ms/step - loss: 0.0230 - binary_accuracy: 0.9943 - categorical_accuracy: 0.4515 - val_loss: 0.1193 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3810
Epoch 2/5
6000/6000 [==============================] - 250s 42ms/step - loss: 0.0211 - binary_accuracy: 0.9950 - categorical_accuracy: 0.4497 - val_loss: 0.1210 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3840
Epoch 3/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0195 - binary_accuracy: 0.9957 - categorical_accuracy: 0.4508 - val_loss: 0.1225 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3860
Epoch 4/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0179 - binary_accuracy: 0.9962 - categorical_accuracy: 0.4523 - val_loss: 0.1241 - val_binary_accuracy: 0.9677 - val_categorical_accuracy: 0.3840
Epoch 5/5
6000/6000 [==============================] - 249s 42ms/step - loss: 0.0166 - binary_ac